In [ ]:
# --- Install Required Libraries ---
!pip install -q bitsandbytes accelerate transformers datasets peft pymupdf



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# --- Mount Google Drive ---
from google.colab import drive
# drive.mount('drive/MyDrive/gc_talks/')

# --- Imports ---
import os
import fitz  # PyMuPDF
import glob
import re
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model



In [ ]:
# --- Custom Metadata Extractors ---
def extract_pdf_with_metadata(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        metadata = doc.metadata
        text = "".join(page.get_text() for page in doc)
        doc.close()
        return {
            "text": text.strip(),
            "source": os.path.basename(pdf_path),
            "author": metadata.get("author", "Unknown"),
            "title": metadata.get("title", os.path.basename(pdf_path)),
            "date": metadata.get("creationDate", "Unknown")
        }
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return None

def extract_txt_with_custom_metadata(txt_path):
    try:
        with open(txt_path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        idx = next((i for i, line in enumerate(lines) if "2:3" in line), None)
        if idx is not None and idx + 2 < len(lines):
            title = lines[idx + 1].strip().strip("\"“”")
            author = lines[idx + 2].strip().replace("By ", "").strip()
            text = "".join(lines[idx + 3:])
        else:
            title, author = "Untitled", "Unknown"
            text = "".join(lines)

        return {
            "text": text.strip(),
            "source": os.path.basename(txt_path),
            "author": author,
            "title": title,
            "date": "Unknown"
        }

    except Exception as e:
        print(f"Error reading {txt_path}: {e}")
        return None

# --- Load Documents with Metadata ---
folder_path = "drive/MyDrive/gc_talks/"
documents = []



In [ ]:
for filepath in glob.glob(folder_path + "*.pdf"):
    doc = extract_pdf_with_metadata(filepath)
    if doc: documents.append(doc)

for filepath in glob.glob(folder_path + "*.txt"):
    doc = extract_txt_with_custom_metadata(filepath)
    if doc: documents.append(doc)

print(f"Loaded {len(documents)} documents.")

# --- Clean, Chunk, and Attach Metadata ---
def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()

def chunk_text(text, max_chunk_size=2000):
    return [text[i:i+max_chunk_size] for i in range(0, len(text), max_chunk_size)]

all_chunks = []
for doc in documents:
    chunks = chunk_text(clean_text(doc["text"]))
    for chunk in chunks:
        prompt = f"""Instruction:
Write in the literary style of {doc['author']}, based on a document titled \"{doc['title']}\" written on {doc['date']}.

Response:
{chunk}
"""
        all_chunks.append({"text": prompt})

# --- Create Dataset and Tokenize ---
dataset = Dataset.from_list(all_chunks).train_test_split(test_size=0.1)

model_name = "google/gemma-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

def tokenize_function(examples):
    outputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    outputs["labels"] = outputs["input_ids"].copy()
    return outputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# --- Load Model and Apply LoRA ---
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()



Loaded 6065 documents.


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

trainable params: 6,422,528 || all params: 8,544,103,424 || trainable%: 0.0752


In [ ]:
# --- Training Configuration ---
training_args = TrainingArguments(
    output_dir="drive/MyDrive/gemma_lora_outputs/",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    learning_rate=2e-4,
    save_steps=50,
    save_total_limit=2,
    logging_steps=50,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    # evaluation_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)



No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# --- Begin Training ---
# trainer.train()

trainer.train(resume_from_checkpoint=True)

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
3200,1.309100
3250,1.353600
3300,1.285100
3350,1.310600
3400,1.289500
3450,1.334300
3500,1.302000
3550,1.307200
3600,1.334700
3650,1.314700


TrainOutput(global_step=4194, training_loss=0.3256199881754208, metrics={'train_runtime': 17893.0258, 'train_samples_per_second': 3.752, 'train_steps_per_second': 0.234, 'total_flos': 1.5996700774335775e+18, 'train_loss': 0.3256199881754208, 'epoch': 2.999553132540888})

In [ ]:
# --- Save Final Adapter and Tokenizer ---
model.save_pretrained("/content/drive/MyDrive/gemma_lora_outputs/final_lora_adapter")
tokenizer.save_pretrained("/content/drive/MyDrive/gemma_lora_outputs/final_tokenizer")


('/content/drive/MyDrive/gemma_lora_outputs/final_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/gemma_lora_outputs/final_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/gemma_lora_outputs/final_tokenizer/tokenizer.model',
 '/content/drive/MyDrive/gemma_lora_outputs/final_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/gemma_lora_outputs/final_tokenizer/tokenizer.json')